# Bridget Hansen - Web Scraping and Cleaning

In [14]:
pip install -U webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser

import time
import random

## Running the Webscraping and Crawling

In [21]:
browser = webdriver.Chrome()
browser.maximize_window()

# Empty lists for data
state_names = []
county_names = []
trump_votes = []
clinton_votes = []

# Visit the main URL
url = "https://www.nytimes.com/elections/2016/results/president"
browser.get(url)
time.sleep(2)

# Find all links inside the footer
state_links = browser.find_elements(By.XPATH, '//ul[@class="eln-page-list"]/li/a')

# Get each state url and store in empty list address
state_urls = [link.get_attribute('href') for link in state_links]
address = []
for url in state_urls:
    address.append(url)

#Scrape for all states excluding alaska and district of columbia
for link in [address[0], address[2],address[3],address[4],address[5],address[7],address[9],address[10],address[11],address[12],address[13],address[14],address[15],address[16],address[17],address[18],address[20],address[22],address[23],address[24],address[25],address[26],address[27],address[28],address[30],address[31],address[32],address[33],address[34],address[35],address[36],address[37],address[38],address[40],address[41],address[42],address[43],address[44],address[46],address[47],address[48],address[49],address[50]]:
    browser.get(link)
    time.sleep(random.uniform(1, 3))

    #Extracting the state name from the state url
    state_name = link.split("/")[-1]

    #Finding the table with county level table
    tables = browser.find_elements(By.XPATH, '//table[@class="eln-county-table"]')
    
    #Clicking the show all counties button if present
    try:
        show_all_button = browser.find_element(By.CLASS_NAME, "eln-toggle-counties")
        show_all_button.click()
        time.sleep(1)
    except:
        pass

    #Locating the county level table on each state link
    for table in tables:
        headers = table.find_elements(By.TAG_NAME, 'th')
        header_texts = [h.text.strip().lower() for h in headers]
        
        #Finding only the presidental election results table
        if 'trump' in header_texts and 'clinton' in header_texts:

            #Finding each row
            state_rows = table.find_elements(By.TAG_NAME, 'tr')[1:]

            #Extracting the cell data for each row, county name, trump votes, clinton votes
            for row in state_rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 3:
                    county_names.append(cells[0].text)
                    trump_votes.append(cells[1].text)
                    clinton_votes.append(cells[2].text)
                    state_names.append(state_name)

#Saving the webscraped data to a dataframe
election_results = pd.DataFrame({
    "state": state_names,
    "county": county_names,
    "trump votes": trump_votes,
    "clinton votes": clinton_votes
})
    
    
# Closing the WebDriver
browser.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=136.0.7103.93)
Stacktrace:
0   chromedriver                        0x0000000104d083bc cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x0000000104d00684 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x000000010483dfbc cxxbridge1$string$len + 90140
3   chromedriver                        0x0000000104818798 chromedriver + 133016
4   chromedriver                        0x00000001048ad048 cxxbridge1$string$len + 544936
5   chromedriver                        0x00000001048c5b50 cxxbridge1$string$len + 646064
6   chromedriver                        0x00000001048793f8 cxxbridge1$string$len + 332888
7   chromedriver                        0x0000000104ccc7e0 cxxbridge1$str$ptr + 2585200
8   chromedriver                        0x0000000104ccfab0 cxxbridge1$str$ptr + 2598208
9   chromedriver                        0

## Displaying the Scraped Data Frame and Doing Data Cleaning

In [7]:
#Displaying the data frame to verify web scraping 
election_results

NameError: name 'election_results' is not defined

In [25]:
#Do value counts for each 50 stats to verify number of counties
election_results['state'].value_counts()

state
texas             254
georgia           159
virginia          133
kentucky          120
missouri          115
kansas            105
illinois          102
north-carolina    100
iowa               99
tennessee          95
nebraska           93
indiana            92
ohio               88
minnesota          87
michigan           83
mississippi        82
oklahoma           77
arkansas           75
wisconsin          72
pennsylvania       67
alabama            67
florida            67
south-dakota       66
louisiana          64
colorado           64
new-york           62
california         58
montana            56
west-virginia      55
north-dakota       53
south-carolina     46
idaho              44
washington         39
oregon             36
new-mexico         33
utah               29
maryland           24
wyoming            23
new-jersey         21
nevada             17
arizona            15
hawaii              4
delaware            3
Name: count, dtype: int64

In [2]:
#Checking data types of the data frame
election_results.dtypes

NameError: name 'election_results' is not defined

In [15]:
#Changing the data type of votes from object to integar 
election_results['trump votes'] = election_results['trump votes'].str.replace(',', '').astype(int)
election_results['clinton votes'] = election_results['clinton votes'].str.replace(',', '').astype(int)

#Changning state and county data type from object to category
election_results["state"] = election_results["state"].astype("category")
election_results["county"] = election_results["county"].astype("category")

In [17]:
#Rechecking data types to verify conversion 
election_results.dtypes

state            category
county           category
trump votes         int64
clinton votes       int64
dtype: object

## Getting a dervived column for Total Votes and Vote Percentage

In [19]:
#Creating a dervived column for total votes and vote percentage 
election_results["total votes"] = election_results["trump votes"] + election_results["clinton votes"]
election_results["trump vote %"] = (election_results["trump votes"] / election_results["total votes"] * 100).round(2)
election_results["clinton vote %"] = (election_results["clinton votes"] / election_results["total votes"] * 100).round(2)

In [21]:
election_results

,state,county,trump votes,clinton votes,total votes,trump vote %,clinton vote %
0,alabama,Jefferson,134768,156873,291641,46.21,53.79
1,alabama,Mobile,95116,72186,167302,56.85,43.15
2,alabama,Madison,89520,62822,152342,58.76,41.24
3,alabama,Shelby,73020,22977,95997,76.06,23.94
4,alabama,Montgomery,34003,58916,92919,36.59,63.41
...,...,...,...,...,...,...,...
3039,wyoming,Crook,3348,273,3621,92.46,7.54
3040,wyoming,Washakie,2911,532,3443,84.55,15.45
3041,wyoming,Weston,3033,299,3332,91.03,8.97
3042,wyoming,Hot Springs,1939,400,2339,82.90,17.10


In [23]:
#Checking data type of the dervived columns
election_results.dtypes

state             category
county            category
trump votes          int64
clinton votes        int64
total votes          int64
trump vote %       float64
clinton vote %     float64
dtype: object

In [25]:
#Checking for duplicates
duplicate_rows = election_results.duplicated()
print(f"Duplicates: {duplicate_rows.sum()}")

Duplicates: 0


## Savinig the Cleaned Web Scraped Data to a CSV

In [27]:
#Saving the data frame to a csv file
election_results.to_csv("election_results_complete.csv", index=False,encoding='utf-8')

In [23]:
election_results  = pd.read_csv('election_results_complete.csv')